# Spatial Point Pattern Analysis in Python

This notebook will demonstrate how to apply methods for Point Patter Analysis using different Python modules. It is based on data from http://insideairbnb.com/, a website that regularly scrapes AirBnB to enable analytics that AirBnB itself does not provide. 

###### Load some libraries

In [ ]:
import pandas as pd # tabular data in Python
import geopandas as gpd # extension to Pandas to work with geodata
import urllib.request # download from the web
import os.path # work with the local file system
from shapely.geometry import Point # basic functions to work with vector geometries
import matplotlib as mpl # plotting
from matplotlib import pyplot as plt # some matplotlib convenience functions

# make figures larger
plt.rcParams['figure.figsize'] = [16, 16]

#### Download some data

Let's download data for the Copenhagen area at the end of May 2020 and end of June 2016 from http://insideairbnb.com/get-the-data.html. The goal will be to check whether we can see some kind of development in the AirBnB listings and their spatial distribution.

For both of the files, we will first check whether they have been downloaded already; if not, download them. Start with 2020:

In [ ]:
data2024 = 'Data/AirBnB-CPH-30-12-2024.csv'
url   = 'https://data.insideairbnb.com/denmark/hovedstaden/copenhagen/2024-12-30/visualisations/listings.csv'
if(os.path.isfile(fname)):
    print(fname,'already downloaded.')
else:
    print('Downloading', fname)
    urllib.request.urlretrieve(url, fname)
    print('Done', fname)

#### Load the data

We'll load the data as a geodataframe first to do some filtering

In [ ]:
df = pd.read_csv(data2024)
listings24 = gpd.GeoDataFrame(df,
    crs=('epsg:4326'), # set the coordinate reference system to WGS84 lat/lon
    geometry=[Point(xy) for xy in zip(df.longitude, df.latitude)]) # construct the point geometries from lat/lon

listings24.head()

In [ ]:
listings20.plot()

Repeat for 2016:

In [ ]:
data2016 = 'Data/AirBnB-CPH-28-06-2016.csv'

df = pd.read_csv(data2016)
listings16 = gpd.GeoDataFrame(df,
    crs=('epsg:4326'), # set the coordinate reference system to WGS84 lat/lon
    geometry=[Point(xy) for xy in zip(df.longitude, df.latitude)]) # construct the point geometries from lat/lon

listings16.head()

In [ ]:
listings16.plot()

Not easy to see any difference between 2024 and 2016, right? That's where point pattern analysis comes in! 

#### Filter data

But first, let's filter the data a bit. We'll only look at listings where the whole place is for rent (not just a room inside):

In [ ]:
listings16["room_type"] == "Entire home/apt"

In [ ]:
print(len(listings16)) # how many listings before filtering?
listings16 = listings16[listings16["room_type"] == "Entire home/apt"]
print(len(listings16)) # and after?
listings16.head()

Next, we'll remove places with less than 25 reviews - we are only interested in AirBnBs that get booked a lot:

In [ ]:
print(len(listings16)) # how many listings before filtering?
listings16 = listings16[listings16["number_of_reviews"] > 25]
print(len(listings16)) # and after?
listings16.head()

Let's do the same for 2020 (in one go):

In [ ]:
print(len(listings24)) # how many listings before filtering?
listings24 = listings24[(listings24["room_type"] == "Entire home/apt") & 
                        (listings24["number_of_reviews"] > 25)]
print(len(listings24))
listings24.head()

### Reproject

A lot of what we are going to do is going to be based on Euclidian distance. Since lat/lon coordinates are spherical (i.e. they are in degrees, describing a position on a sphere), we'll project the coordinates to a planar coordinate reference system – [UTM Zone 32 North](https://epsg.io/25832), specifically:

In [ ]:
print(listings16.crs)
print(listings24.crs)

listings16 = listings16.to_crs(epsg=25832)
listings24 = listings24.to_crs(epsg=25832)

print(listings16.crs)
print(listings24.crs)

In [ ]:
listings16.head()

Notice how the numbers in the geometry column are now different? They have been transformed to UTM32N!

# Mean centre

In Geopandas, we can extract the x and y components of the coordinates like so:

In [ ]:
listings16.geometry.x

So we can easily take the means and turn them into a new point:

In [ ]:
mean_centre_16 = Point((listings16.geometry.x.mean(), listings16.geometry.y.mean()))
mean_centre_24 = Point((listings24.geometry.x.mean(), listings24.geometry.y.mean()))

Let's show them on a map. First, load the outline of Copenhagen municipality (plus Frederiksberg) as a reference:

In [ ]:
fname = 'CPH.geojson'
url   = 'https://gist.github.com/crstn/bec5259ac8602bbc8961ab6dfc18e5cf/raw/489d981ec22dda1b13e676f4662732e7a139504e/CPH.geojson'

if(os.path.isfile(fname)):
    print(fname,'already downloaded.')
else:
    print('Downloading', fname)
    urllib.request.urlretrieve(url, fname)
    print('Done', fname)

cph = gpd.read_file(fname)
cph.plot()

Also project this one to UTM32N:

In [ ]:
cph = cph.to_crs(epsg=25832)
cph.crs

Plotting is easier when we turn the mean centres into Geodataframes:

In [ ]:
mean_centre_16_gdf = gpd.GeoDataFrame(crs=('epsg:25832'), geometry=[mean_centre_16])
mean_centre_24_gdf = gpd.GeoDataFrame(crs=('epsg:25832'), geometry=[mean_centre_24])

Btw, this is one way to make a simlple map with multiple layers with Geopandas and Matplotlib:

In [ ]:
base = cph.plot(color='white', edgecolor='black')
mean_centre_16_gdf.plot(ax=base, marker='o', color='red', markersize=25);
mean_centre_24_gdf.plot(ax=base, marker='x', color='blue', markersize=25);

Not very useful. Let's check out whether the AirBnBs are more concentrated in the center in 2024:

# Standard distance

In [ ]:
standardDistance16 = (listings16.distance(mean_centre_16)).mean()
standardDistance16

In [ ]:
standardDistance24 = (listings16.distance(mean_centre_24)).mean()
standardDistance24

Almost the same... we can still plot them, though:

In [ ]:
standardCircle16 = mean_centre_16_gdf.buffer(standardDistance16)
standardCircle24 = mean_centre_24_gdf.buffer(standardDistance24)

Same plot as before, but with the circle:

In [ ]:
base = cph.plot(color='white', edgecolor='black')

mean_centre_16_gdf.plot(ax=base, marker='o', color='red', markersize=25);
standardCircle16.plot(ax=base, color='none', edgecolor='red');

mean_centre_24_gdf.plot(ax=base, marker='x', color='blue', markersize=25);
standardCircle24.plot(ax=base, color='none', edgecolor='blue');

# Intensity

We'll calculate the events (AirBnBs, in this case) per aereal unit. To do that, we first need to know how large the study area is:

In [ ]:
area_cph = cph.area[0]
area_cph

These are square meters (because UTM is in meters) in scientific notation, let's convert it to square kilometers:

In [ ]:
area_cph = area_cph / (1000 * 1000)
print(area_cph)

Now simply devide the number of listings by that area:

In [ ]:
intensity16 = len(listings16) / area_cph
print(intensity16)

intensity24 = len(listings24) / area_cph
print(intensity24)

So the intensity _did_ decrease. But how are the events distributed spatially? Let's visualize some quadrats:

In [ ]:
p = plt.hist2d(listings16.geometry.x, listings16.geometry.y, bins=50)
plt.colorbar()
plt.show()

In [ ]:
p = plt.hist2d(listings24.geometry.x, listings24.geometry.y, bins=50)
plt.colorbar()
plt.show()

# Quadrat statistic

Let's check out quadrat-based methods using complete spatial randomness (CSR). We'll only do this for the 2020 data now.

In [ ]:
import libpysal as ps
import numpy as np
from pointpats import PointPattern, Window, as_window
from pointpats.distance_statistics import k
from pointpats import PoissonPointProcess as csr
import pointpats.quadrat_statistics as qs

Pointpats needs to create a specifc PointPattern object, which we'll creatre from our AirBnB locations:

In [ ]:
airbnb_points_24 = np.array([event for event in zip(listings24.geometry.x, listings24.geometry.y)])
airbnb_points_24

Turn into point pattern and plot:

In [ ]:
pp_24 = PointPattern(points = airbnb_points_24, 
                  window=Window([list(x.exterior.coords) for x in cph.geometry[0].geoms]))

pp_24.plot(window=True,title='AirBnBs 2024')

In [ ]:
pp_24.summary()

Now we can generate the quadrat statistic:

In [ ]:
q_r = qs.QStatistic(pp_24, shape= "rectangle" ,nx = 20, ny = 20)
q_r.plot()

Now we'll compare that with 10 completely random point patterns within our window: 

_(You would normally do more, e.g. 100 or even 1000, but that would take quite long with this number of points.)_

### Let's take a break while this is running.

In [ ]:
csr_process = csr(pp_24.window, pp_24.n, 10, asPP=True)

Let's take a look at one of the randomly generated point patterns:

In [ ]:
csr_process.realizations[0].plot()

In [ ]:
q_r_empirical = qs.QStatistic(pp_24, shape= "rectangle",nx = 20, ny = 20, realizations = csr_process)

In [ ]:
print(q_r_empirical.chi2)
print(q_r_empirical.chi2_realizations)
print(q_r_empirical.chi2_pvalue)

We can safely disregard the null hypothesis that the AirBnBs are not clustered.

## Kernel Density Estimation

Link to interactive KDE tutorial: https://mathisonian.github.io/kde/

In [ ]:
import seaborn as sns

Documentation for seaborn's KDE plot function: https://seaborn.pydata.org/generated/seaborn.kdeplot.html

In [ ]:
sns.kdeplot(x=listings24.geometry.x, 
            y=listings24.geometry.y, 
            fill=True, 
            thresh=0, 
            levels=50,
            cmap="magma",
            bw_adjust = 0.5)

Play around with the ```bw_adjust``` parameter!

We can also plot the KDE together with other layers for context:

In [ ]:
f, ax = plt.subplots(1, figsize=(16, 16))

sns.kdeplot(x=listings24.geometry.x, 
            y=listings24.geometry.y, 
            fill=True,             
            thresh=0, 
            levels=50,
            cmap="Purples",
            bw_adjust = .4,
            ax=ax);

cph.plot(edgecolor='grey', color="none", linewidth=0.8, ax=ax)

ax.set_axis_off()
plt.axis('equal')
plt.show()

### Average nearest neighbor distance

Run the following block a couple of times. What happens here?

In [ ]:
Do = pp_24.mean_nnd
De = csr(pp_24.window, pp_24.n, 1, asPP=True).realizations[0].mean_nnd

print('NND =', str(Do), '/', str(De), '=', str(Do/De))

### Task

Repeat the above analyses for the 2016 data. What changes do you observe from 2016 to 2024?